In [1]:
import numpy as np
#np.set_printoptions(threshold=np.inf)

world = -1. * np.ones((18,14))
starts = [(17,3), (17,4), (17,5), (17,6)]
goals = [(1,13), (2,13), (3,13), (4,13), (5,13)]

beg_ = [(6,11),(7,9),(8,8),(9,8),(10,8),(11,8),(12,8),(13,7),(14,7),(15,7),(16,7),(17,7)]
end_ = [(0,13),(1,4),(2,0),(3,0),(4,0),(5,0),(6,1),(7,1),(8,1),(9,2),(10,2),(11,3),(12,3),(13,3),(14,3),(15,2),(16,2),(17,2)]

for coup in end_:
    world[coup[0], 0:coup[1]+1] = -10
    
for coup in beg_:
    world[coup[0], coup[1]:14] = -10
   
for cell in goals:
    world[cell] = -1

print(world)

[[-10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10. -10. -10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10. -10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1. -10. -10. -10.]
 [-10. -10.  -1.  -1.  -1.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10.]
 [-10. -10.  -1.  -1.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10.  -1.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10.  -1.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10. -10.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10. -10.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10. -10.  -1.  -1.  -1. -10. -10. -10.

In [10]:
MIN = 1
MAX = 5

def generate_actions():
    actions = []
    for i in range(MAX+1):
        for j in range(MAX+1):
            if MIN <= (i + j) <= MAX:
                actions.append((i,j))
    return actions

actions = generate_actions()

def move(state, action):
    new_state = state.copy()
    
    if action[0] == 0:
        pass
    elif action[1] == 0:
        pass
    else:
        if action[0] <= action[1]:
            # Only diagonal movement
            pass
        else:
            # Diagonal plus ahead
            pass
    

def monte_carlo():
    Q = {}
    
    for _ in range(1000):
        # Start episode 
        alive = True

        state = starts[np.random.randint(0,len(starts))]

        while alive:
            
            action = actions[np.random.randint(0,len(starts))]
            
        break